# UDF

As UDFs (User-Defined Functions) no PySpark são funções personalizadas que você pode criar para aplicar operações complexas a colunas em um DataFrame. Elas permitem estender a funcionalidade do PySpark para realizar transformações de dados personalizadas.

Aqui está um exemplo de como criar e usar uma UDF no PySpark:

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Inicialize uma sessão Spark
spark = SparkSession.builder.appName("ExemploUDF").getOrCreate()

# Crie um DataFrame de exemplo
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["Nome", "Idade"])

# Defina uma função Python que será usada como UDF
def dobrar_idade(idade):
    return idade * 2

# Registre a função Python como uma UDF
dobrar_idade_udf = udf(dobrar_idade, IntegerType())

# Aplique a UDF à coluna 'Idade' e crie uma nova coluna 'IdadeDobrada'
df = df.withColumn("IdadeDobrada", dobrar_idade_udf(df["Idade"]))

# Mostre o DataFrame resultante
df.show()
```

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [4]:
%%sh
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [2]:
df = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/air_cia/")
)

df.printSchema()

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)



In [5]:
from unidecode import unidecode

air_cia = (
    df
    .toDF(*[unidecode(a) for a in df.columns])
)

air_cia.printSchema()

root
 |-- razao_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aereas: string (nullable = true)
 |-- endereco_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisao_operacional: string (nullable = true)
 |-- data_decisao_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- iata: string (nullable = true)



In [12]:
(
    air_cia
    .write
    .format("parquet")
    .mode("append")
    .save("/home/app/data/1.bronze/air_cia/")
)

Py4JJavaError: An error occurred while calling o75.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:288)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.apa

In [5]:
 def remove_caracters_special(string):
    if string == None:
        return None
    else:
        return unidecode(string)

In [6]:
from pyspark.sql.types import StringType

convert_name_udf = F.udf(remove_caracters_special, StringType())

# Udf com decorator

In [9]:
from pyspark.sql.functions import *

@F.udf (returnType = StringType())
def cnpj_transforming(string):
    if string:
        return string.replace('.','').replace('/','').replace('-','')
    else:
        return None

@udf (returnType = StringType())
def telefone_transforming(string):
    if string:
        return string.replace('(','').replace(')','').replace(' ','').replace('-','').replace("|Fax:","/")
    else:
        return None

air_cia_final =(
    air_cia
    .withColumn("cnpj", cnpj_transforming(F.col("cnpj")))
    .withColumn("telefone", F.split(telefone_transforming(F.col('telefone')),"/"))
    .withColumn("endereco_sede", convert_name_udf(F.col('endereco_sede')))
    .withColumn("decisao_operacional", convert_name_udf(F.col('decisao_operacional')))
    .withColumn("atividades_aereas", convert_name_udf(F.col('atividades_aereas')))
)
air_cia_final.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------
 razao_social             | ABSA - AEROLINHAS BRASILEIRAS S.A.                                                                                           
 icao_iata                | LTG M3                                                                                                                       
 cnpj                     | 00074635000133                                                                                                               
 atividades_aereas        | TRANSPORTE AEREO REGULAR                                                                                                     
 endereco_sede            | AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA SANTOS DUMONT, KM 66, SISTEMA VIARIO PRINCIPAL, S/ No, 13.052-970, CAMPINAS-SP 
 telefone                 | [1155828055]                                    

In [10]:
air_cia_final = air_cia_final.drop_duplicates(["razao_social","icao_iata","cnpj"])
air_cia_final = air_cia_final.dropna()

In [11]:
air_cia_final.dtypes

[('razao_social', 'string'),
 ('icao_iata', 'string'),
 ('cnpj', 'string'),
 ('atividades_aereas', 'string'),
 ('endereco_sede', 'string'),
 ('telefone', 'array<string>'),
 ('e-mail', 'string'),
 ('decisao_operacional', 'string'),
 ('data_decisao_operacional', 'string'),
 ('validade_operacional', 'string'),
 ('icao', 'string'),
 ('iata', 'string')]

In [15]:
air_cia_final.where(F.col("data_decisao_operacional") == '22/04/2015' ).show()

+--------------------+---------+--------------+--------------------+--------------------+------------+--------------+-------------------+------------------------+--------------------+----+----+
|        razao_social|icao_iata|          cnpj|   atividades_aereas|       endereco_sede|    telefone|        e-mail|decisao_operacional|data_decisao_operacional|validade_operacional|icao|iata|
+--------------------+---------+--------------+--------------------+--------------------+------------+--------------+-------------------+------------------------+--------------------+----+----+
|ABSA - AEROLINHAS...|   LTG M3|00074635000133|TRANSPORTE AEREO ...|AEROPORTO INTERNA...|[1155828055]|gar@tam.com.br|      DECISAO No 41|              22/04/2015|          23/04/2025| LTG|  M3|
+--------------------+---------+--------------+--------------------+--------------------+------------+--------------+-------------------+------------------------+--------------------+----+----+

